### Installation

In [ ]:
!pip install -q unsloth

In [3]:
!pip uninstall -y xformers

Found existing installation: xformers 0.0.32.post1
Uninstalling xformers-0.0.32.post1:
  Successfully uninstalled xformers-0.0.32.post1


In [4]:
!pip check

unsloth 2025.10.1 requires xformers, which is not installed.
vllm 0.11.0 requires xformers, which is not installed.


### Unsloth

In [5]:
from unsloth import PatchDPOTrainer

PatchDPOTrainer()

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


2025-10-12 19:45:17.731478: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


INFO 10-12 19:45:24 [__init__.py:216] Automatically detected platform cuda.
WARNING 10-12 19:45:25 [interface.py:381] Using 'pin_memory=False' as WSL is detected. This may slow down the performance.
🦥 Unsloth Zoo will now patch everything to make training faster!


In [ ]:
from modelscope.hub.snapshot_download import snapshot_download
model_id = 'unsloth/llama-3-8b-Instruct'
local_dir = './models/llama3-8b'

snapshot_download(repo_id=model_id, cache_dir=local_dir)

'./models/llama3-8b/unsloth/llama-3-8b-Instruct'

In [1]:
from unsloth import FastLanguageModel

max_seq_length = 512
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "models/llama3-8b/unsloth/llama-3-8b-Instruct",
    max_seq_length = max_seq_length,
    load_in_4bit = True,
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


2025-10-12 20:31:06.369193: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


INFO 10-12 20:31:14 [__init__.py:216] Automatically detected platform cuda.
WARNING 10-12 20:31:15 [interface.py:381] Using 'pin_memory=False' as WSL is detected. This may slow down the performance.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.10.1: Fast Llama patching. Transformers: 4.56.2. vLLM: 0.11.0.
   \\   /|    NVIDIA GeForce RTX 5060 Ti. Num GPUs = 1. Max memory: 15.928 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.8.0+cu128. CUDA: 12.0. CUDA Toolkit: 12.8. Triton: 3.4.0
\        /    Bfloat16 = TRUE. FA [Xformers = None. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

<a name="Data"></a>
### Data Prep

We first download the data files.

In [5]:
!git clone https://gitlab.com/lchengtw/ML2025Spring-HW7.git

Cloning into 'ML2025Spring-HW7'...
remote: Enumerating objects: 9, done.
remote: Total 9 (delta 0), reused 0 (delta 0), pack-reused 9 (from 1)
Receiving objects: 100% (9/9), 8.63 KiB | 982.00 KiB/s, done.
Resolving deltas: 100% (1/1), done.


Then, we load the json file here.

In [3]:
import json

with open("ML2025Spring-HW7/train.json", 'r') as jsonfile:
    full_data = json.load(jsonfile)

with open("ML2025Spring-HW7/test.json", 'r') as jsonfile:
    test_data = json.load(jsonfile)

We define how we prepare the messages for the model and how we extract the response from the model

In [4]:
import re

def data_formulate(data):
    messages = [
        {"role": "system", "content": "Your entire response must be 100 characters or less."},
        {"role": "user", "content": data['prompt']},
    ]
    prompt = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    return prompt

def extract_assistant_response(text):
    try:
        # Split by assistant header marker
        parts = text.split("<|start_header_id|>assistant<|end_header_id|>")
        if len(parts) < 2:
            return None

        # Split by end of text marker
        assistant_part = parts[1]
        response_parts = assistant_part.split("<|eot_id|>")

        # Clean up any whitespace
        return response_parts[0].strip()
    except Exception as e:
        print(f"Error extracting assistant response: {e}")
        return None

Let's observe how the model responses before aligning it.

In [10]:
original_model_response = []
for data in test_data:
    id = data['id']
    prompt = data['prompt']
    print(f'\nQuestion {id}: {prompt}')
    inputs = data_formulate(data)
    outputs = model.generate(
        **tokenizer(inputs, return_tensors = "pt").to("cuda"),
        max_new_tokens = 128,
        do_sample=False
    )
    output = tokenizer.batch_decode(outputs)[0]
    output = extract_assistant_response(output)
    original_model_response.append(output)
    print()
    print(output)


Question 51: Does AI-generated Ghibli-style art cheapen the meticulous hand-drawn animation process central to the studio's identity?

Yes, AI-generated Ghibli-style art may diminish the unique charm and character of traditional hand-drawn animation, which is a hallmark of Studio Ghibli's identity.

Question 52: Should museums and art galleries include AI-generated Ghibli-style art in exhibitions about animation history?

Yes, museums and art galleries can include AI-generated Ghibli-style art in exhibitions about animation history to showcase the evolution of animation techniques and the role of AI in creative processes.

Question 53: Does AI-generated Ghibli-style art create confusion about authorship and artistic voice?

Yes, AI-generated Ghibli-style art can raise questions about authorship and artistic voice, as it blurs the line between human and machine creativity.

Question 54: Can AI-made art that looks like Studio Ghibli movies show the same deep feelings that the real Ghibl

Now we preapre the data for aligning.

Please adjust the parameters here to complete the observations for the assignment.

In [11]:
# TODO: Adjust the parameters here
num_epoch = 3
data_size = 50
support_ratio = 0

In [12]:
#### DO NOT CHANGE ####

from datasets import Dataset

# Select part of the data for training
training_data = full_data[:data_size]

# Define the size of the support dataset
support_data_size = int(data_size * support_ratio)

# Prepare the data for the training dataset
prompt_list = [data_formulate(data) for data in training_data]
chosen_list = [data['support'] for data in training_data[:support_data_size]] + [data['oppose'] for data in training_data[support_data_size:]]
rejected_list = [data['oppose'] for data in training_data[:support_data_size]] + [data['support'] for data in training_data[support_data_size:]]

# Create the training dataset
train_dataset = Dataset.from_dict({'prompt': prompt_list, 'chosen': chosen_list, 'rejected': rejected_list})

Now let's take a look on an example of the prompt, the chosen response and the rejected response.

In [13]:
prompt_list[0]

"<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nYour entire response must be 100 characters or less.<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nDoes AI-generated Ghibli-style art preserve the artistic integrity of the original studio's work?<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\n"

In [14]:
chosen_list[0]

'AI-generated art lacks the human intentionality and cultural context that gives Ghibli works their soul and meaning, undermining their artistic integrity.'

In [15]:
rejected_list[0]

"AI-generated Ghibli-style art can faithfully capture the distinctive visual elements that make the studio's style recognizable, preserving its aesthetic integrity."

We now add LoRA adapters so we only need to update 1 to 10% of all parameters.

Please do not change anything here.

In [16]:
#### DO NOT CHANGE ####

model = FastLanguageModel.get_peft_model(
    model,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],

    r = 16,           # Larger = higher accuracy, but might overfit
    lora_alpha = 16,  # Recommended alpha == r at least
    lora_dropout = 0.1,
    bias = "none",
    random_state = 3407, # Do not modify the random_state for reproducibility
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth: Dropout = 0 is supported for fast patching. You are using dropout = 0.1.
Unsloth will patch all other layers, except LoRA matrices, causing a performance hit.
Unsloth 2025.10.1 patched 32 layers with 0 QKV layers, 0 O layers and 0 MLP layers.


<a name="Train"></a>
### Train the DPO model

Now we define the trainer.

Please (also) do not change anything here.

In [17]:
#### DO NOT CHANGE ####

from transformers import TrainingArguments
from trl import DPOTrainer, DPOConfig
from unsloth import is_bfloat16_supported

dpo_trainer = DPOTrainer(
    model = model,
    ref_model = None,
    args = DPOConfig(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_ratio = 0.1,
        num_train_epochs = num_epoch,
        learning_rate = 1e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "paged_adamw_8bit",
        weight_decay = 0.0,
        lr_scheduler_type = "linear",
        seed = 42,
        output_dir = "outputs",
        report_to = "none",
    ),
    beta = 0.1,
    train_dataset = train_dataset,
    tokenizer = tokenizer,
)

Extracting prompt in train dataset (num_proc=16):   0%|          | 0/50 [00:00<?, ? examples/s]

Applying chat template to train dataset (num_proc=16):   0%|          | 0/50 [00:00<?, ? examples/s]

Tokenizing train dataset (num_proc=16):   0%|          | 0/50 [00:00<?, ? examples/s]

Now we start training!

In [18]:
dpo_trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 50 | Num Epochs = 3 | Total steps = 21
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 41,943,040 of 8,072,204,288 (0.52% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss,rewards / chosen,rewards / rejected,rewards / accuracies,rewards / margins,logps / chosen,logps / rejected,logits / chosen,logits / rejected,eval_logits / chosen,eval_logits / rejected,nll_loss
1,0.693100,0.000000,0.000000,0.000000,0.000000,-73.869385,-69.687378,-0.747039,-0.795452,0,0,0
2,0.693100,0.000000,0.000000,0.000000,0.000000,-74.825729,-66.795479,-0.906720,-0.873787,No Log,No Log,No Log
3,0.691300,0.016510,0.012497,0.500000,0.004013,-78.021980,-65.852051,-0.762486,-0.679316,No Log,No Log,No Log
4,0.662700,0.056049,-0.006833,0.875000,0.062881,-68.875443,-71.668030,-0.762881,-0.719778,No Log,No Log,No Log
5,0.582700,0.115534,-0.123206,1.000000,0.238740,-83.971725,-69.518997,-0.841511,-0.742139,No Log,No Log,No Log
6,0.533100,0.208618,-0.154497,0.875000,0.363115,-73.813370,-66.017876,-0.870376,-0.781186,No Log,No Log,No Log
7,0.275100,0.404324,-0.767289,1.000000,1.171614,-71.953094,-70.880035,-0.726372,-0.923195,No Log,No Log,No Log
8,0.111400,0.991311,-1.377444,1.000000,2.368755,-61.895863,-74.914375,-0.802369,-0.828241,No Log,No Log,No Log
9,0.152400,1.201814,-0.861432,1.000000,2.063246,-61.545700,-82.595001,-0.780302,-0.712868,No Log,No Log,No Log
10,0.082100,1.340153,-1.640133,1.000000,2.980285,-69.415070,-83.072968,-0.814492,-0.831572,No Log,No Log,No Log


TrainOutput(global_step=21, training_loss=0.22815745589988573, metrics={'train_runtime': 72.5498, 'train_samples_per_second': 2.068, 'train_steps_per_second': 0.289, 'total_flos': 0.0, 'train_loss': 0.22815745589988573, 'epoch': 3.0})

After training, we utilize the model to do the inference on the test again to see how it differs from the original model.

In [19]:
aligned_model_response = []
for data in test_data:
    id = data['id']
    prompt = data['prompt']
    print(f'\nQuestion {id}: {prompt}')
    inputs = data_formulate(data)
    outputs = model.generate(
        **tokenizer(inputs, return_tensors = "pt").to("cuda"),
        max_new_tokens = 128,
        do_sample=False
    )
    output = tokenizer.batch_decode(outputs)[0]
    output = extract_assistant_response(output)
    aligned_model_response.append(output)
    print()
    print(output)


Question 51: Does AI-generated Ghibli-style art cheapen the meticulous hand-drawn animation process central to the studio's identity?

Yes, AI-generated Ghibli-style art fundamentally contradicts the studio's ethos and values, which prioritize human craftsmanship and attention to detail.

Question 52: Should museums and art galleries include AI-generated Ghibli-style art in exhibitions about animation history?

No.

Question 53: Does AI-generated Ghibli-style art create confusion about authorship and artistic voice?

Yes, AI-generated Ghibli-style art perpetuates the problem of authorship and artistic voice, as it misrepresents the creative process and undermines the value of human labor and intentionality.

Question 54: Can AI-made art that looks like Studio Ghibli movies show the same deep feelings that the real Ghibli films do?

No, AI-generated art lacks the human experience, intention, and emotional depth that Ghibli films convey.

Question 55: Does limiting AI from generating Gh

Next, we save the results in .json for your NTU COOL submission.

Please note that this is designed for Colab, you may have to change the directory name for other machines.

In [ ]:
student_id = "B12345678" # TODO: fill in your student id here.
dir_name = "/content" # TODO: If you use machines other than colab, please adjust the directory here.
# Do NOT change the following for this block.
file_name = f"{dir_name}/{student_id}_hw7_epoch{num_epoch}_ratio{support_ratio}_size{data_size}.json"
output_list = []
for data in test_data:
  original_response = original_model_response.pop(0)
  aligned_response = aligned_model_response.pop(0)
  output_list.append({"id": data["id"], "prompt": data["prompt"], "original_response": original_response, "aligned_response": aligned_response})
output_data = {"num_epoch": num_epoch, "data_size": data_size, "support_ratio": support_ratio, "results": output_list}
with open(file_name, "w") as output_file:
    json.dump(output_data, output_file, indent=4)


Finally, we provide code for free testing.

You may freely adjust the system prompt, user prompt and generate settings here for model behavior observations.

In [15]:
def make_prompt(system, prompt):
    messages = [
        {"role": "system", "content": system},
        {"role": "user", "content": prompt},
    ]
    prompt = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    return prompt

# TODO: Try your system prompt and user prompt here.
system = "Your entire response must be 100 characters or less."
prompt = "Is it ethical for AI to generate One Piece-style art?"

inputs = make_prompt(system, prompt)
outputs = model.generate(
    **tokenizer(inputs, return_tensors = "pt").to("cuda"),
    max_new_tokens = 512, # TODO: You may use this for early stop.
    do_sample=False, # Please keep this to False and do not tweak other parameters.
)
output = tokenizer.batch_decode(outputs)[0]
output = extract_assistant_response(output)
print(output)

It's ethical for AI to generate One Piece-style art, as long as the AI doesn't claim ownership or profit from the art without permission from the original creators.


And that's it for homework 7! If you have any questions, please consider posting questions in the discussion forum first so all the classmates can benefit. TAs will also prioritize responding to questions posted there.

Also, please make sure that you have completed the submission for both GradeScope and NTU Cool.

Good luck!
